# Marktstammdatenregister
https://www.marktstammdatenregister.de/MaStR/Datendownload

In [1]:
import os
import re
import xml.etree.ElementTree as ET

In [2]:
# params
dir = 'data/Marktstammdatenregister'

features = ['AnlagenEegSolar', 'AnlagenEegWind', 'EinheitenSolar']

In [3]:
def extract_sort_key(filepath):
    match = re.search(r'/(.+?)_(\d+)\.xml$', filepath)
    if match:
        category = match.group(1)
        number = int(match.group(2)) 
    else:
        category = filepath.split('/')[-1]
        number = float('inf')
    return category, number

def read_xml(file_path):
    """
    Liest eine XML-Datei ein und gibt den Wurzelknoten zurück.
    
    :param file_path: Pfad zur XML-Datei
    :return: Wurzel des XML-Baums (ElementTree.Element)
    """
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        print(f"XML-Datei erfolgreich eingelesen: {file_path}")
        return root
    except ET.ParseError as e:
        print(f"Fehler beim Einlesen der XML-Datei: {e}")
        return None
    except FileNotFoundError:
        print(f"Datei nicht gefunden: {file_path}")
        return None
    
def print_xml_structure(element, level=0):
    """
    Gibt die Struktur eines XML-Baums rekursiv aus.
    
    :param element: XML-Element
    :param level: Aktuelle Ebene der Rekursion (für Einrückungen)
    """
    indent = "  " * level
    print(f"{indent}Tag: {element.tag}, Attribute: {element.attrib}")
    for child in element:
        print_xml_structure(child, level + 1)
        

def find_non_zero_attributes(element):
    """
    Findet alle Attribute in einem XML-Baum, deren Werte nicht 0 sind.
    
    :param element: XML-Element
    """
    for child in element:
        non_zero_attrs = {k: v for k, v in child.attrib.items() if v != 0}
        if non_zero_attrs:
            print(f"Tag: {child.tag}, Nicht-Null-Attribute: {non_zero_attrs}")
        find_non_zero_attributes(child)

In [4]:
file_names = os.listdir(dir)
paths = [os.path.join(dir, file) for file in file_names]
paths = [path for path in paths if any(feature in path for feature in features)]
paths = sorted(paths, key=extract_sort_key)

In [ ]:
file = paths[50]

root = read_xml(file)

In [ ]:
print_xml_structure(root)